<a href="https://colab.research.google.com/github/mahmoudafandi/Deep-learning-3/blob/main/graph_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install torch torchvision torchaudio
!pip install torch-geometric




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
import torch.nn as nn

# Load Cora dataset
dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0]


Processing...
Done!


In [7]:
class GATNet(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8):
        super(GATNet, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=0.6)
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=0.6)
        # Optional linear layer
        self.linear = nn.Linear(out_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.gat1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.gat2(x, edge_index)
        x = self.linear(x)
        return x


In [8]:
def train(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            val_acc = evaluate(model, data, mask=data.val_mask)
            print(f"Epoch {epoch}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}")


In [9]:
def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out[mask].argmax(dim=1)
        acc = (pred == data.y[mask]).sum() / mask.sum()
    return acc.item()

def test(model, data):
    test_acc = evaluate(model, data, mask=data.test_mask)
    print(f"Test Accuracy: {test_acc:.4f}")


In [10]:
# Instantiate model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GATNet(dataset.num_node_features, 8, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Train the model
train(model, data, optimizer, criterion)

# Evaluate on test set
test(model, data)


Epoch 10, Loss: 1.5529, Val Acc: 0.4760
Epoch 20, Loss: 0.8001, Val Acc: 0.6740
Epoch 30, Loss: 0.3431, Val Acc: 0.7260
Epoch 40, Loss: 0.0994, Val Acc: 0.7440
Epoch 50, Loss: 0.0165, Val Acc: 0.7400
Epoch 60, Loss: 0.0057, Val Acc: 0.7320
Epoch 70, Loss: 0.0040, Val Acc: 0.7300
Epoch 80, Loss: 0.0040, Val Acc: 0.7280
Epoch 90, Loss: 0.0046, Val Acc: 0.7280
Epoch 100, Loss: 0.0052, Val Acc: 0.7260
Epoch 110, Loss: 0.0056, Val Acc: 0.7280
Epoch 120, Loss: 0.0057, Val Acc: 0.7280
Epoch 130, Loss: 0.0055, Val Acc: 0.7300
Epoch 140, Loss: 0.0052, Val Acc: 0.7260
Epoch 150, Loss: 0.0050, Val Acc: 0.7260
Epoch 160, Loss: 0.0048, Val Acc: 0.7220
Epoch 170, Loss: 0.0046, Val Acc: 0.7220
Epoch 180, Loss: 0.0044, Val Acc: 0.7220
Epoch 190, Loss: 0.0043, Val Acc: 0.7220
Epoch 200, Loss: 0.0041, Val Acc: 0.7200
Test Accuracy: 0.7580
